In [5]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os 
from time import time
from time import sleep
import pandas as pd

In [ ]:
#To check whether the code works to pull the data from API

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '4e883b7d-eb83-43b9-a490-e1eb92e81ae9',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [9]:
#display the type of data
type(data)

dict

In [11]:
#Fully display the columns and rows of the output to see entire output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
#Data we pulled is in JSON format
#Convert the JSON data into dataframe
#Add a new column Timestamp to see time of pull
df = pd.json_normalize(data['data'])
df['Timestamp'] = pd.to_datetime('now')
df

/var/folders/b3/pjc9lb1j2ws7v0fcx8z9tgc80000gn/T/ipykernel_70988/852868870.py:2: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df['Timestamp'] = pd.to_datetime('now')


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,Timestamp
0,1,Bitcoin,BTC,bitcoin,10815,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.962075e+07,19620750,...,51.0319,9.013469e+11,NaN,2024-02-07T07:40:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-02-07 07:46:34.597931
1,1027,Ethereum,ETH,ethereum,8384,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.201822e+08,120182175.858488,...,17.2040,2.839067e+11,NaN,2024-02-07T07:40:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-02-07 07:46:34.597931
2,825,Tether USDt,USDT,tether,74242,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,9.618140e+10,99600626149.840363,...,5.8235,9.951925e+10,NaN,2024-02-07T07:39:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2024-02-07 07:46:34.597931
3,1839,BNB,BNB,bnb,2053,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.495461e+08,149546056.588637,...,2.7384,4.519094e+10,NaN,2024-02-07T07:39:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-02-07 07:46:34.597931
4,5426,Solana,SOL,solana,625,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.367482e+08,568875404.46949,...,2.5284,5.434654e+10,NaN,2024-02-07T07:40:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-02-07 07:46:34.597931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,28069,Scrooge,SCROOGE,scrooge-new,1,2023-09-20T17:32:56.000Z,[],NaN,0.000000e+00,1000000000000000,...,0.0000,1.343901e+06,NaN,2024-02-07T07:39:00.000Z,1839.0,BNB Smart Chain (BEP20),BNB,bnb,0x9dfee72aea65dc7e375d50ea2bd90384313a165a,2024-02-07 07:46:34.597931
4996,23043,Y2K,Y2K,y2k,16,2022-12-21T11:06:37.000Z,[arbitrum-ecosytem],20000000.0,0.000000e+00,20000000,...,0.0000,7.244458e+06,NaN,2024-02-07T07:39:00.000Z,11841.0,Arbitrum,ARB,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,2024-02-07 07:46:34.597931
4997,7414,Behodler,EYE,beholder,3,2020-10-15T00:00:00.000Z,[],NaN,0.000000e+00,10000000,...,0.0000,1.705560e+06,NaN,2024-02-07T07:40:00.000Z,1027.0,Ethereum,ETH,ethereum,0x155ff1A85F440EE0A382eA949f24CE4E0b751c65,2024-02-07 07:46:34.597931
4998,12229,DogeGF,DOGEGF,dogegf,9,2021-09-30T19:49:03.000Z,"[memes, doggone-doggerel]",NaN,0.000000e+00,34538000000000000,...,0.0000,4.519933e+06,NaN,2024-02-07T07:39:00.000Z,1027.0,Ethereum,ETH,ethereum,0xfb130d93e49dca13264344966a611dc79a456bc5,2024-02-07 07:46:34.597931


### To automate the process of data pulling from the API it is better to write the code as a function
`Pulling_data` function pull the data from the API and store in dataframe by appending into it. Each time 15 data points is pulled from the API. This API provider have a limit of 333.

In [9]:
def pulling_data():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
    'start':'1',
    'limit':'15',
    'convert':'USD'
    }
    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '4e883b7d-eb83-43b9-a490-e1eb92e81ae9',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
    #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    df2 = pd.json_normalize(data['data'])
    df2['Timestamp'] = pd.to_datetime('now')
    df = df.append(df2)


### Automatically run the function for 333 times(that is the limit). After each pull it sleeps for 60s.

In [ ]:
for i in range(333):
    pulling_data()
    print('API Runner completed')
    sleep(60) #sleep for 1 minute
exit()


In [11]:
df.to_csv('/Users/devikasunil/Documents/GitHub/Cryto API using Python/Crypto_data_pulled.csv')